# ECE 239 Final Project


## Import libraries and data

In [23]:
import numpy as np
import tensorflow as tf
from tensorflow import keras


In [24]:
import numpy as np
X_test = np.load("./data/X_test.npy")
y_test = np.load("./data/y_test.npy")
person_train_valid = np.load("./data/person_train_valid.npy")
X_train_valid = np.load("./data/X_train_valid.npy")
y_train_valid = np.load("./data/y_train_valid.npy")
person_test = np.load("./data/person_test.npy")


### Shape of data

In [25]:
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))


Training/Valid data shape: (2115, 25, 1000)
Test data shape: (443, 25, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


### Taking out EOG (last 3)

In [26]:
X_train_valid = X_train_valid[:,0:22,:]
X_test = X_test[:,0:22,:]
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))

Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)


## Separate train into train + validation

In [27]:
#TODO: Check this
percent_validation = 0.1
num_training, EEGs, samples_per_example = X_train_valid.shape
indices = np.arange(num_training)
np.random.shuffle(indices)
validation_indices = indices[:int(percent_validation*num_training)]
training_indices = indices[int(percent_validation*num_training):]
X_train = X_train_valid[training_indices,:,:]
y_train = y_train_valid[training_indices]
X_validation = X_train_valid[validation_indices,:,:]
y_validation = y_train_valid[validation_indices]

print ('Training data shape: {}'.format(X_train.shape))
print ('Training target shape: {}'.format(y_train.shape))
print ('Validation data shape: {}'.format(X_validation.shape))
print ('Validation target shape: {}'.format(y_validation.shape))




Training data shape: (1904, 22, 1000)
Training target shape: (1904,)
Validation data shape: (211, 22, 1000)
Validation target shape: (211,)


In [28]:

#Note y data is number, convert it to labels
print (y_validation[33])
print (y_validation.shape)
print (y_test.shape)

def convert_to_label(num):
    return np.array([1.0*(num==769),  1.0*(num==770), 1.0*(num==771), 1.0*(num==772)])

y_train_labels = np.array([convert_to_label(yi) for yi in y_train])
y_validation_labels = np.array([convert_to_label(yi) for yi in y_validation])
y_test_labels = np.array([convert_to_label(yi) for yi in y_test])

print (y_validation_labels[33])
print (y_validation_labels.shape)
print (y_test_labels.shape)


770
(211,)
(443,)
[0. 1. 0. 0.]
(211, 4)
(443, 4)


## Naive approach: without RNN

In [39]:
#### test model
model = keras.Sequential()


model.add(keras.layers.Conv1D(3, 3, strides=2, padding='same', data_format='channels_last', use_bias=True, input_shape=(22,1000)))
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.MaxPooling1D(pool_size=2, strides=None, padding='same', data_format='channels_last'))
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.Conv1D(3, 3, strides=2, padding='same', data_format='channels_last', use_bias=True))
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.MaxPooling1D(pool_size=2, strides=None, padding='same', data_format='channels_last'))
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(500))
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.Dense(150))
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.Dense(100))
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.Dense(4))
model.add(keras.layers.Activation('softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train_labels, validation_data=(X_validation, y_validation_labels), epochs=5)

scores = model.evaluate(X_test, y_test_labels, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])




Train on 1904 samples, validate on 211 samples
Epoch 1/5
1904/1904 [==============================] - 1s 591us/sample - loss: 1.4239 - acc: 0.2600 - val_loss: 1.4216 - val_acc: 0.2559
Epoch 2/5
1904/1904 [==============================] - 1s 300us/sample - loss: 1.3755 - acc: 0.3030 - val_loss: 1.4471 - val_acc: 0.2654
Epoch 3/5
1904/1904 [==============================] - 1s 299us/sample - loss: 1.3225 - acc: 0.3566 - val_loss: 1.4203 - val_acc: 0.3033
Epoch 4/5
1904/1904 [==============================] - 1s 348us/sample - loss: 1.2614 - acc: 0.3923 - val_loss: 1.4242 - val_acc: 0.2891
Epoch 5/5
443/443 [==============================] - 0s 248us/sample - loss: 1.5017 - acc: 0.3273
Test loss: 1.5016825123780468
Test accuracy: 0.32731378
